In [6]:
# Installing dependencies
import numpy as np
import pymc as pm

In [14]:
# Generate some toy data
np.random.seed(42)
# n data point in the data set
n = 100
# x in [0, 10]
x_data = np.linspace(0, 10, int(n))
# the known parameters of the linear model (y=a*x+b)
true_slope = a = 3
true_intercept = b = 1.0
# y=a*x+b+N(mu=0,sigma=1)
## Note: np.random.normal adds Gaussian noise from N(0,1)
y_data = true_slope * x_data + true_intercept + np.random.normal(size=n)

In [37]:
# Define the model
with pm.Model() as model:
    # Register the data - useful for later predictions,
    # when we replace x to predict y
    x = pm.Data('x', x_data, mutable=True)
    y = pm.Data('y_obs', y_data)

    # Define priors
    # Slope
    a = pm.Normal("slope", mu=0, sigma=1)
    # Intercapt
    b = pm.Normal("intercept", mu=0, sigma=1)
    # Standard deviation - Note HalfNormal!
    s = pm.HalfNormal("sigma", sigma=1, observed=y)
    # Define the likelihood (note the "observed" argument, and mu=ax+b)
    likelihood = pm.Normal("y", mu=a*x + b, sigma=s)
    # Now we define the inference engine
    # We will sample from the posterior using MCMC (Hamiltonian MC, NUTS)
    step = pm.NUTS()
    # The trace variable contains the samples a,b,s ~ P(a,b,s|D)
    trace = pm.sample(1000, tune=100, init=None, step=step, chains=2)

Multiprocess sampling (2 chains in 4 jobs)
NUTS: [slope, intercept, y]


Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 19 seconds.
The acceptance probability does not match the target. It is 0.9106, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9134, but should be close to 0.8. Try to increase the number of tuning steps.
